In [10]:
from model import UserBasedCF, ItemBasedCF, MatrixFactorizationCF
from helper_functions import load_data, load_config

In [2]:
# Train the models on the training data
def train_model(reviews_dir, game_details_path, selection_file, selection_name=""):
   
    train_data, _ = load_data(reviews_dir, game_details_path, selection_file)
    
    # Load models
    model_dir = 'data/model'
    
    user_based_model = UserBasedCF()
    user_based_model.fit(train_data)
    user_based_model.save(f'{model_dir}/user_based_cf_model{selection_name}.pkl')
    
    item_based_model = ItemBasedCF()
    item_based_model.fit(train_data)
    item_based_model.save(f'{model_dir}/item_based_cf_model{selection_name}.pkl')
    
    matrix_factorization_model = MatrixFactorizationCF()
    matrix_factorization_model.fit(train_data)
    matrix_factorization_model.save(f'{model_dir}/matrix_factorization_cf_model{selection_name}.pkl')

In [11]:
config = load_config()
for variation, params in config.items():
    train_model(
        params['reviews_dir'],
        params['game_details_path'],
        params['selection_file'],
        params['selection_name'])

SchemaError: type String is incompatible with expected type Null